# Salary prediction by vacancy description

## Dataset description

The dataset represents the data about vacancies which were published in the world net for different countries. The vacancy info has a full description of this vacancy, title, location, company, working category, salary etc.
In this assignment you have to predict the possibility of raising the salary threshold, using the vacancy description. The data is presented in the dataframe. The columns of interest are:
* FullDescription - description of vacancy
* SalaryNormalized - predicted salary threshold.

Dataset description

There are steps which are required to successfully complete the assignment:
1. Read dataset
2. Perform text transformation by removing punctuation terms and stop words.
3. Generate n-grams.	
4. Count TF * IDF features
5. Fit model for generated features.


## Reading dataset

Init pyspark session

In [1]:
from __future__ import division, print_function, unicode_literals # For the compatibility with Python 2

In [2]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder\
                            .enableHiveSupport()\
                            .appName("spark sql")\
                            .master("local[4]")\
                            .getOrCreate()

In [3]:
sc=spark_session.sparkContext

In [4]:
#!ls /data/vacancie


In [5]:
#!ls /data/covertype2

In [6]:
train_data = spark_session.read.csv("/data/covertype2/train.csv",inferSchema=True,header=True)

#train_data.printSchema()


In [7]:
#dataset = train_data.select('FullDescription','SalaryNormalized')
#train_data.select('Hillshade_3pm').show()


## Transforming dataset

In [8]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer


stringIndexer = StringIndexer(inputCol="Wild_Type", outputCol="Wild_Type_Indexed")
model = stringIndexer.fit(train_data)
indexed = model.transform(train_data)
encoder = OneHotEncoder(inputCol="Wild_Type_Indexed", outputCol="Wild_Type_Hot")
encoded = encoder.transform(indexed)


stringIndexer = StringIndexer(inputCol="Soil_Type", outputCol="Soil_Type_Indexed")
model = stringIndexer.fit(encoded)
raw_data = model.transform(encoded)
encoder = OneHotEncoder(inputCol="Soil_Type_Indexed", outputCol="Soil_Type_Hot")
encoded = encoder.transform(raw_data)

cols = list(set(encoded.columns) - {'Wild_Type', 'Soil_Type', 'Target', 'Soil_Type_Indexed', 'Wild_Type_Indexed' })
print(len(cols))

#encoded = encoded.select(cols)


12


In [9]:
from pyspark.ml.feature import StandardScaler

from  pyspark.ml.feature  import VectorAssembler

# Define assembler 
assembler = VectorAssembler(
    inputCols=cols,
    outputCol='features')

# transform
vector_indexed_data = assembler.transform(encoded)
vector_indexed_data.printSchema()

dataset = vector_indexed_data.select('features', 'Target')

# scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
#                         withStd=True, withMean=False)

# # Compute summary statistics by fitting the StandardScaler
# scaler_model = scaler.fit(dataset)
# scaledData = scaler_model.transform(dataset)
# dataset = scaledData

root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Wild_Type: string (nullable = true)
 |-- Soil_Type: string (nullable = true)
 |-- Target: integer (nullable = true)
 |-- Wild_Type_Indexed: double (nullable = true)
 |-- Wild_Type_Hot: vector (nullable = true)
 |-- Soil_Type_Indexed: double (nullable = true)
 |-- Soil_Type_Hot: vector (nullable = true)
 |-- features: vector (nullable = true)



In [10]:
dataset.show(5, False)


+-------------------------------------------------------------------------------------------------------------+------+
|features                                                                                                     |Target|
+-------------------------------------------------------------------------------------------------------------+------+
|(52,[0,1,2,3,4,7,44,45,47,49,50,51],[195.0,266.0,245.0,3122.0,451.0,1.0,433.0,3069.0,1.0,188.0,10.0,75.0])   |1     |
|(52,[0,1,2,3,4,5,44,45,46,49,50,51],[177.0,308.0,229.0,3018.0,4546.0,1.0,60.0,5359.0,1.0,192.0,15.0,14.0])   |1     |
|(52,[0,1,2,3,4,5,44,45,46,49,50,51],[236.0,151.0,240.0,3146.0,1371.0,1.0,541.0,5887.0,1.0,132.0,12.0,-2.0])  |2     |
|(52,[0,1,2,3,4,5,44,45,46,49,50,51],[226.0,163.0,242.0,2980.0,1087.0,1.0,553.0,3538.0,1.0,149.0,6.0,21.0])   |2     |
|(52,[0,1,2,3,4,11,44,45,46,49,50,51],[220.0,187.0,250.0,2972.0,4119.0,1.0,255.0,6390.0,1.0,158.0,16.0,109.0])|2     |
+-----------------------------------------------

# Fitting model

Split the dataset to train and validation part (it is better to use 90% for the train part and 10% for the validation part)

In [11]:
train_data, test_data = dataset.randomSplit([0.9, 0.1])

Fit the Logistic Regression to the model on the splitted train part. Use about 15 iterations for the training process.

<b>Hint.</b> Use regularization parameter in order to prevent overfitting.

In [12]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

In [13]:
#lr = LogisticRegression(featuresCol='features',labelCol='labels')

lr = rf = RandomForestClassifier(labelCol="Target", featuresCol="features", numTrees=100, maxBins=40, maxDepth=7)

model = lr.fit(dataset)


In [14]:
#model.featureImportances

Print the loss function for each iteration. What can you notice from the behaviour of loss function?

<b>Hint.</b> Use summary.objectiveHistory for this case.

In [15]:
# trainingSummary = model.summary

# trainingSummary.predictions.show(5)

Apply the model to the validation set

In [16]:
# test_results = model.evaluate(test_data)

# test_results.predictions.show(5)

#test_data.show()

In [17]:
#model.summary.objectiveHistory

In [18]:
predictions = model.transform(test_data)

#predictions.show(3)

Calculate AUC-ROC for the predicted data. For this purpose, you can use BinaryClassificationEvaluator from ml.evaluation model.

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


evaluator = MulticlassClassificationEvaluator(labelCol='Target') #default=label not labels
evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})

0.7029411136504411

<b>Self-check question:</b>

1. Try to fit and predict model using pure words. Has the result changed?



# Performing test submission

Apply the learned models to the test dataset.

<b>Note!</b> The test dataset will be changed during the test phase. Your last cell output must be the output of the AUC-ROC score.

In [20]:
train_data = spark_session.read.csv("/data/covertype2/train.csv",inferSchema=True,header=True)

test_data_ = spark_session.read.csv("/data/covertype2/test.csv",inferSchema=True,header=True)



stringIndexer = StringIndexer(inputCol="Wild_Type", outputCol="Wild_Type_Indexed")
modeler = stringIndexer.fit(train_data)  # :) ATTENTION 
indexed = modeler.transform(test_data_)
encoder = OneHotEncoder(inputCol="Wild_Type_Indexed", outputCol="Wild_Type_Hot")
encoded = encoder.transform(indexed)


stringIndexer = StringIndexer(inputCol="Soil_Type", outputCol="Soil_Type_Indexed")
modeler = stringIndexer.fit(train_data)  # :) ATTENTION 
raw_data = modeler.transform(encoded)
encoder = OneHotEncoder(inputCol="Soil_Type_Indexed", outputCol="Soil_Type_Hot")
encoded = encoder.transform(raw_data)

cols = list(set(encoded.columns) - {'Wild_Type', 'Soil_Type', 'Target', 'Soil_Type_Indexed', 'Wild_Type_Indexed'})

print(len(cols))

# Define assembler 
# Define assembler 
assembler = VectorAssembler(
    inputCols=cols,
    outputCol='features')

# transform
vector_indexed_data = assembler.transform(encoded)
vector_indexed_data.printSchema()

dataset = vector_indexed_data.select('features', 'Target')

dataset.show()
# scaler = StandardScaler(inputCol="features", outputCol="sc

12
root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Wild_Type: string (nullable = true)
 |-- Soil_Type: string (nullable = true)
 |-- Target: integer (nullable = true)
 |-- Wild_Type_Indexed: double (nullable = true)
 |-- Wild_Type_Hot: vector (nullable = true)
 |-- Soil_Type_Indexed: double (nullable = true)
 |-- Soil_Type_Hot: vector (nullable = true)
 |-- features: vector (nullable = true)

+--------------------+------+
|            features|Target|
+--------------------+------+
|(52,[0,1,2,3,4,10...|     3|
|(52,

In [21]:
# Transform dataset and calculate auc-roc
dataset.show(3,False)
predictions_2 = model.transform(dataset)


+------------------------------------------------------------------------------------------------------------+------+
|features                                                                                                    |Target|
+------------------------------------------------------------------------------------------------------------+------+
|(52,[0,1,2,3,4,10,44,45,48,49,50,51],[197.0,44.0,125.0,2025.0,216.0,1.0,95.0,124.0,1.0,30.0,40.0,75.0])     |3     |
|(52,[0,1,2,3,4,5,44,45,46,49,50,51],[226.0,52.0,197.0,3092.0,2408.0,1.0,182.0,4425.0,1.0,98.0,18.0,39.0])   |2     |
|(52,[0,1,2,3,4,7,44,45,47,49,50,51],[188.0,346.0,213.0,3403.0,1254.0,1.0,698.0,2505.0,1.0,163.0,16.0,111.0])|1     |
+------------------------------------------------------------------------------------------------------------+------+
only showing top 3 rows



In [22]:
evaluator.evaluate(predictions_2, {evaluator.metricName: "accuracy"})

0.7052542460735698